# Création

In [1]:
import chromadb

# Création du client et de la base locale
client = chromadb.PersistentClient(path="./chroma_db")

# Création d'une collection
detection = client.get_or_create_collection("detection")
Image = client.get_or_create_collection("image")

ModuleNotFoundError: No module named 'chromadb'

# Fonctions

In [ ]:
# Ajout de documents avec vecteurs + métadonnées
def add_detection(ids, embeddings, metadatas):
    detection.add(
        ids=ids,
        embeddings=embeddings,
        metadatas=metadatas
    )

In [3]:
def del_detection(ids):
    detection.delete(ids=ids)

In [4]:
# Génération de données de test (10 vecteurs de dimension 384)
import numpy as np
import random

def generate_test_data(n_vectors=10, dim=384):
    vectors = np.random.rand(n_vectors, dim)
    metadatas = [{"category": random.choice(["cat", "dog", "bird"])} for _ in range(n_vectors)]
    return vectors, metadatas

In [5]:
# Recherche par proximité de vecteurs
from scipy.spatial.distance import cosine
import numpy as np

def ask_query(collection, query_vector, query, max_dist):   #max_dist compris entre [0,2]
    # Obtenir le nombre de documents dans la collection
    num_documents = len(collection.get()["ids"])

    # Recherche hybride (similitude vectorielle + filtre sur la catégorie)
    results = detection.query(
        query_embeddings=[query_vector],
        n_results=num_documents,  # Nombre de résultats souhaités
        where=query,  # Filtrage par métadonnée
        include=["embeddings", "metadatas", "distances"],
    )

    filtered_results = {    # Filtrer les résultats pour n'afficher que ceux avec une distance inférieure à x
            "ids": [],
            "embeddings": [],
            "metadatas": [],
            "distances": []
        }
    
    cosine_dist = []

    for i in range(len(results["ids"])):
        embedding = np.squeeze(np.array(results["embeddings"][i]))  # Convertir en vecteur 1D
        query_vector = np.squeeze(np.array(query_vector))  # Assurer que la requête est aussi 1D
        
        print(f"Embedding shape: {embedding.shape}, Query shape: {query_vector.shape}")  # Debug
        
        cosine_dist.append(cosine(embedding, query_vector))  # Calcul de la distance cosinus

    

    for i in range(len(results["ids"])):
        if cosine_dist[i] <= max_dist:  # Access the first element of the list
            filtered_results["ids"].append(results["ids"][i])
            filtered_results["embeddings"].append(results["embeddings"][i])
            filtered_results["metadatas"].append(results["metadatas"][i])
            filtered_results["distances"].append(results["distances"][i][0])  # Access the first element of the list

    results = filtered_results

    return results


In [ ]:
# Recherche par proximité de vecteurs
from scipy.spatial.distance import cosine
import numpy as np

def ask_query(collection, query_vector, query=None, max_dist=1):   #max_dist compris entre [0,2]


    all_data = collection.get(include=["embeddings", "metadatas"])

    # Obtenir le nombre de documents dans la collection
    num_documents = len(all_data["ids"])
    
    # Calcul de la distance cosinus entre le vecteur de requête et tous les vecteurs de la collection
    cosine_dist = [cosine(query_vector, np.squeeze(np.array(embedding))) for embedding in all_data["embeddings"]]

    # Filtrage des résultats
    results = {
        "ids": [],
        "embeddings": [],
        "metadatas": [],
        "distances": []
    }

    if query != None:
        query = query
        for i in range(num_documents):
            if (cosine_dist[i] <= max_dist) & (all_data["metadatas"][i]["category"] == query["category"]):
                results["ids"].append(all_data["ids"][i])
                results["embeddings"].append(all_data["embeddings"][i])
                results["metadatas"].append(all_data["metadatas"][i])
                results["distances"].append(cosine_dist[i])
    else:
        for i in range(num_documents):
            if cosine_dist[i] <= max_dist:
                results["ids"].append(all_data["ids"][i])
                results["embeddings"].append(all_data["embeddings"][i])
                results["metadatas"].append(all_data["metadatas"][i])
                results["distances"].append(cosine_dist[i])

    return results, cosine_dist

    

In [ ]:
import cv2
import os
import datetime
from ultralytics import YOLO
from sentence_transformers import SentenceTransformer
import shutil

FPS_TARGET = 24  # FPS cible
TARGET_SIZE = (640, 360 )  # Nouvelle taille des frames

# Charger le modèle YOLO
model = YOLO("yolo11n.pt")

# Clear cache and re-download the SentenceTransformer model
cache_dir = os.path.expanduser("~/.cache/huggingface/hub")
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)

# Charger le modèle d'embeddings
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def create_unique_folder(base_dir, prefix="frames"):
    timestamp = datetime.datetime.now().strftime("%d%m_%H%M%S")
    folder_path = os.path.join(base_dir, f"{prefix}_{timestamp}")
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

def resize_frame(frame, size):
    return cv2.resize(frame, size, interpolation=cv2.INTER_AREA)

def extract_and_resize_frames(video_path, output_dir, target_fps, target_size, start_frame=0, custom_output_dir=None):
    if not os.path.exists(video_path):
        print(f"Erreur: Le fichier vidéo '{video_path}' n'existe pas.")
        return None

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Erreur: Impossible d'ouvrir la vidéo '{video_path}'")
        return None

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames_original = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_frames_target = int((total_frames_original / original_fps) * target_fps)

    print(f"Total frames original: {total_frames_original}")
    print(f"Total frames cible: {total_frames_target}")

    # Vérification et ajustement de start_frame
    if start_frame > total_frames_target:
        print(f"Erreur: La frame de départ ({start_frame}) est supérieure au nombre total de frames cibles ({total_frames_target}).")
        start_frame = total_frames_target  # Ajuster start_frame pour ne pas dépasser total_frames_target
        print(f"start_frame ajusté à {start_frame}")

    # Produit en croix pour ajuster la frame de départ
    start_time = (start_frame * total_frames_original) / total_frames_target
    adjusted_start_frame = round(start_time)
    print(f"Start frame ajusté: {adjusted_start_frame}")

    # Utilisation du dossier personnalisé ou création d'un nouveau dossier
    frames_dir = custom_output_dir if custom_output_dir else create_unique_folder(output_dir, "frames_resized")
    os.makedirs(frames_dir, exist_ok=True)

    # Extraire et redimensionner les frames à partir de la frame ajustée
    extracted_count = start_frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, adjusted_start_frame)  # Placer le lecteur vidéo à la frame ajustée

    while True:
        success, frame = cap.read()
        if not success:
            break

        resized_frame = resize_frame(frame, target_size)
        frame_filename = os.path.join(frames_dir, f"frame_{extracted_count:06d}.png")
        cv2.imwrite(frame_filename, resized_frame)
        extracted_count += 1

    cap.release()
    print(f"Extraction et redimensionnement terminés. {extracted_count - adjusted_start_frame} images enregistrées dans '{frames_dir}'")
    return frames_dir


def create_video_from_frames(frames_dir, output_path, fps):
    frames = sorted(f for f in os.listdir(frames_dir) if f.endswith(".png"))
    if not frames:
        print(f"Erreur: Aucune frame trouvée dans '{frames_dir}'")
        return
    
    first_frame = cv2.imread(os.path.join(frames_dir, frames[0]))
    height, width, _ = first_frame.shape
    
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    for frame_filename in frames:
        frame = cv2.imread(os.path.join(frames_dir, frame_filename))
        out.write(frame)
    
    out.release()
    print(f"Vidéo générée à '{output_path}'")

def analyze_frames(frames_dir):
    image_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".tiff")
    image_files = [f for f in os.listdir(frames_dir) if f.lower().endswith(image_extensions)]
    if not image_files:
        print("Aucune image trouvée pour l'analyse.")
        return

    analysed_folder = os.path.join(frames_dir, "analysed_frames")
    os.makedirs(analysed_folder, exist_ok=True)
    
    detections_data = []
    embeddings_list = []
    
    for image_name in image_files:
        image_path = os.path.join(frames_dir, image_name)
        print(f"📷 Analyse de : {image_name}")
        results = model(image_path)

        if results:  # Vérifie si des résultats ont été trouvés
            analysed_image_path = os.path.join(analysed_folder, image_name)
            results[0].save(filename=analysed_image_path)
            print(f"✅ Image annotée sauvegardée : {analysed_image_path}")
        
            for result in results:
                boxes = result.boxes.xywh.cpu().numpy()
                confs = result.boxes.conf.cpu().numpy()
                classes = result.boxes.cls.cpu().numpy()
                class_names = [result.names[int(cls)] for cls in classes]
                
                for i in range(len(boxes)):
                    detection = {
                        "image": image_name,
                        "class": class_names[i],
                        "x": float(boxes[i][0]),
                        "y": float(boxes[i][1]),
                        "width": float(boxes[i][2]),
                        "height": float(boxes[i][3]),
                        "confidence": float(confs[i]),
                    }
                    detections_data.append(detection)
                    text_data = f"{detection['class']} {detection['confidence']}"
                    embedding = embedder.encode(text_data)
                    embeddings_list.append(embedding)
                    print(f"🔹 Objet détecté : {detection['class']} | Confiance : {detection['confidence']:.2f}")
                    add_detection(i, embedding, detection)                 
        else:
            print(f"🚨 Aucune détection sur {image_name}") 
    return embeddings_list

# ============================
#           EXÉCUTION
# ============================

video_path = "C:/Users/mstep/OneDrive/Bureau/Projet_IA/Pre-traitement/Vidéo/Video_2.mp4"
frames_base_dir = "C:/Users/mstep/OneDrive/Bureau/Projet_IA/Pre-traitement/Frames"
timestamp = datetime.datetime.now().strftime("%d%m_%H%M%S")
output_video_path = os.path.join("C:/Users/mstep/OneDrive/Bureau/Projet_IA/Pre-traitement", f"video_output_{timestamp}.mp4")

custom_folder = ""
frames_dir = extract_and_resize_frames(video_path, frames_base_dir, FPS_TARGET, TARGET_SIZE, start_frame=0, custom_output_dir=custom_folder)

if frames_dir:
    vectors = analyze_frames(frames_dir)

    # Correction ici : on s'assure que le dossier d'analyse est bien dans custom_folder
    analysed_folder = os.path.join(frames_dir, "analysed_frames")
    os.makedirs(analysed_folder, exist_ok=True)

    create_video_from_frames(analysed_folder, output_video_path, FPS_TARGET)

# Execution

In [21]:
# Récupérer toutes les données (ids, embeddings, metadatas)
all_data = detection.get(include=["embeddings", "metadatas"])

# Afficher tout le contenu
for i in range(len(all_data["ids"])):
    print(f"🔹 ID: {all_data['ids'][i]}")
    if all_data['embeddings'] is not None:
        print(f"🧠 Embedding: {all_data['embeddings'][i]}")
    print(f"📌 Métadonnées: {all_data['metadatas'][i]}")
    print("-" * 40)

print(len(all_data['ids']))
print(all_data['embeddings'][0].shape)



🔹 ID: 0
🧠 Embedding: [4.20689374e-01 6.65608704e-01 6.88242674e-01 1.07389353e-01
 8.25969994e-01 8.34605634e-01 9.07591522e-01 6.10553741e-01
 4.33776766e-01 6.46125153e-02 2.94450462e-01 2.65345097e-01
 6.07924521e-01 9.31004763e-01 2.87431151e-01 6.21124327e-01
 5.69217503e-01 1.29633561e-01 8.99026752e-01 3.19665134e-01
 5.20852029e-01 1.64124638e-01 1.57231197e-01 5.66328049e-01
 2.84178793e-01 6.23025596e-01 2.57900000e-01 7.06783414e-01
 5.30249536e-01 8.58998418e-01 8.45940337e-02 7.89973378e-01
 1.05879247e-01 2.73048542e-02 9.92994457e-02 6.38203323e-01
 9.70619678e-01 4.88070995e-01 8.53528917e-01 4.13090318e-01
 9.63321090e-01 8.66409481e-01 8.96133542e-01 4.84728336e-01
 3.93009961e-01 6.48832321e-02 5.67146420e-01 9.98340845e-02
 8.98653150e-01 6.69959709e-02 9.97304395e-02 4.66064624e-02
 6.25485301e-01 3.66494268e-01 8.86465907e-02 2.39447162e-01
 6.07931674e-01 1.24731369e-01 9.66137469e-01 3.03407520e-01
 8.09358180e-01 6.34647012e-01 6.78389430e-01 6.38032377e-01
 1.

In [ ]:
# Recherche par Query

research, dist = ask_query(detection, vectors[0], query={"category": "cat"}, max_dist=0.8)

In [24]:
# Afficher les résultats

print(f"Distance: {dist}")
print("-" * 40)

for i in range(len(research["ids"])):
    print(f"🔹 ID: {research['ids'][i]}")
    if research['embeddings'] is not None:
        print(f"🧠 Embedding: {research['embeddings'][i]}")
    print(f"📌 Métadonnées: {research['metadatas'][i]}")
    print(f"📏 Distance: {research['distances'][i]}")
    print("-" * 40)

Distance: [np.float64(0.27231995604661363), np.float64(0.25738199923352834), np.float64(0.25235919332983836), np.float64(0.2599436694536835), np.float64(0.2599322877716268), np.float64(0.25539403415917794), np.float64(0.2445621658644278), np.float64(0.27854107105101156), np.float64(0.2711473299916052), np.float64(0.261722185931446)]
----------------------------------------
🔹 ID: 5
🧠 Embedding: [0.16501515 0.43102512 0.74864864 0.6914652  0.06673695 0.18245603
 0.65888977 0.85647494 0.21840625 0.21164064 0.79739434 0.90714836
 0.17796686 0.20462783 0.49020752 0.24111085 0.13503134 0.90303487
 0.92616206 0.1855838  0.20166236 0.58065093 0.01438745 0.78612912
 0.91387314 0.68796426 0.73462766 0.56113076 0.83777064 0.92407566
 0.76992941 0.21945147 0.60039604 0.4345997  0.8447724  0.1000421
 0.3625446  0.75698316 0.85323173 0.06705854 0.31771275 0.0792671
 0.34102848 0.79042852 0.53134298 0.46995682 0.84930736 0.9365682
 0.59891498 0.2929078  0.54295337 0.43831956 0.15694937 0.8857857
 0.6

# Intégration LLM

In [28]:
import numpy as np
from Requete_LLM import extract_objects, embed_objects

# Exemple d'utilisation
text = "un chien courant après une balle"
objects = extract_objects(text)
embeddings_dict = embed_objects(objects)

print(f"Objets : {objects}")
print(f"Embeddings : {embeddings_dict}")

# Vérifier si des embeddings existent
if embeddings_dict:
    # Convertir les embeddings en tableau NumPy
    embeddings_list = np.array(list(embeddings_dict.values()))
    
    # Calculer la moyenne des embeddings
    query_vector = np.mean(embeddings_list, axis=0).tolist()

    # Exécuter la requête
    results, dist = ask_query(detection, query_vector, max_dist=0.95)

    # Affichage des résultats
    print(f"Distance cosinus : {dist}")
    
    for i in range(len(results["ids"])):
        print(f"🔹 ID: {results['ids'][i]}")
        print(f"🧠 Embedding: {results['embeddings'][i]}")
        print(f"📌 Métadonnées: {results['metadatas'][i]}")
        print(f"📏 Distance: {results['distances'][i]}")
        print("-" * 40)
else:
    print("Aucun embedding trouvé pour cette requête.")


Objets : ['dog', 'ball']
Embeddings : {'dog': [-0.053146976977586746, 0.014194389805197716, 0.00714567257091403, 0.06860867887735367, -0.0784803032875061, 0.010167457163333893, 0.10228320211172104, -0.012064817361533642, 0.09521341323852539, -0.03035016357898712, 0.0021646935492753983, -0.06486446410417557, -0.0025944202207028866, 0.006218955386430025, -0.003928649239242077, -0.030624626204371452, -0.04791149124503136, -0.019300563260912895, -0.059885382652282715, -0.10416741669178009, -0.08614888787269592, 0.03635949268937111, -0.025526097044348717, 0.001638932735659182, -0.0714421197772026, 0.061680007725954056, 0.0171945933252573, -0.0566110797226429, 0.024812210351228714, -0.07782246917486191, -0.032499197870492935, -0.008699050173163414, -0.011532525531947613, 0.038167353719472885, -0.0569307804107666, -0.05327099934220314, 0.004925680346786976, 0.03250066936016083, 0.07253247499465942, 0.03298494592308998, 0.024722937494516373, -0.0833452120423317, -0.015685806050896645, -0.04811